## Imports

In [1]:
import pandas as pd
from statsmodels.stats import multitest
import matplotlib.pyplot as plt
import pandas as pd
from timers import *

print("Done")

Done


## Prepare dataframe

In [2]:
hive_IDs = {"New Left Hive": '6b5cb8b012cb45038eacc24770a2fff7',
            "Utah OW Nuc Left": '13a71f80590a4184a2956058441c3be3',
            "Utah OW Nuc Right": 'dca02aae55b74ca1b1a6ac35042d2254',
           }

Bee_Weather = BeeWeather()
Bee_Weather.set()

ambient = Ambient()
ambient.set()

master_df = pd.DataFrame()

for hive_creds in list(hive_IDs.items()):
    current_hive = Hive(*hive_creds)
    current_hive.set()
    if not current_hive.get_weight().empty:
        master_df = pd.concat([master_df, pd.DataFrame(current_hive.get_weight()).rename(columns={"Weight": hive_creds[0] + "_weight"})], axis = 1)
    if not current_hive.get_upper_temp().empty:
        master_df = pd.concat([master_df, pd.DataFrame(current_hive.get_upper_temp()).rename(columns={"Temperature": hive_creds[0] + "_upper_temp"})], axis = 1)
    if not current_hive.get_lower_temp().empty:
        master_df = pd.concat([master_df, pd.DataFrame(current_hive.get_lower_temp()).rename(columns={"Temperature": hive_creds[0] + "_lower_temp"})], axis = 1)
    if not current_hive.get_humidity().empty:
        master_df = pd.concat([master_df, pd.DataFrame(current_hive.get_humidity()).rename(columns={"Humidity": hive_creds[0] + "_humidity"})], axis = 1)

    break
# display(master_df.sort_values(by=['Unix_Time']))
display(Bee_Weather.get_beeweather(interp = 1))
display(ambient.get_ambient(interp = 1))

Processing Broodminder weather data.
Reading Broodminder weather data.
Processing Ambient data.
Processing Broodminder data.
Reading Broodminder data.


,Unix_Time,Temperature,Precipitation_inches,Relative_Humidity,Cloud_Cover,WindSpeed10mAbove_mph,SurfacePressure_millibars,WindDirection_degrees
1970-01-01 00:00:00,1658728800,76.3,0.00,42.1,0,6.5,868.8,147.6
1970-01-01 00:05:00,1659808800,74.2,0.00,67.1,100,13.9,869.8,326.2
1970-01-01 00:10:00,1660888800,81.0,0.00,35.6,100,5.3,868.3,34.8
1970-01-01 00:15:00,1661968800,90.0,0.00,18.7,0,3.3,873.0,332.3
1970-01-01 00:20:00,1663048800,73.0,0.00,24.7,100,7.2,865.6,95.6
1970-01-01 00:25:00,1664128800,71.2,0.00,26.6,0,3.2,876.8,291.0
1970-01-01 00:30:00,1665208800,56.9,0.00,45.7,0,3.7,874.8,198.8
1970-01-01 00:35:00,1666288800,66.7,0.00,29.3,1,2.0,872.8,261.2
1970-01-01 00:40:00,1667368800,59.6,0.00,23.8,100,19.3,857.8,170.4
1970-01-01 00:45:00,1668448800,33.1,0.00,63.8,24,3.1,876.1,247.8


C:\Users\khard\OneDrive\Documents\GitHub\Creative-Envy-Dashboard\timers.py:122: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return df.resample('5min').interpolate("linear")


,dateutc,winddir,windspeedmph,windgustmph,maxdailygust,tempf,humidity,hourlyrainin,eventrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin,totalrainin,uv,solarradiation,feelsLike,dewPoint,lastRain
1970-01-01 00:00:00,1.651796e+09,252,0.00,0.00,8.05,77.9,66,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,12.11,78.49,65.61,NaN
1970-01-01 00:05:00,1.651886e+09,229,3.36,4.47,23.94,68.5,53,0.0,0.05,0.05,0.05,0.05,0.05,0.05,0,0.00,67.54,50.68,2022-05-06T13:44:00.000Z
1970-01-01 00:10:00,1.651976e+09,217,1.12,3.36,15.88,59.4,77,0.0,0.00,0.03,0.08,0.08,0.08,0.08,0,0.00,59.40,52.19,2022-05-07T09:21:00.000Z
1970-01-01 00:15:00,1.652066e+09,295,0.00,0.00,6.93,56.5,94,0.0,0.00,0.00,0.00,0.08,0.08,0.08,0,0.00,56.50,54.79,2022-05-07T09:21:00.000Z
1970-01-01 00:20:00,1.652156e+09,22,0.00,0.00,0.00,54.7,93,0.0,0.00,0.00,0.00,0.08,0.08,0.08,0,0.00,54.70,52.72,2022-05-07T09:21:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-03 13:10:00,1.721378e+09,144,0.00,0.00,4.47,76.3,42,0.0,0.00,0.00,0.00,0.00,15.88,15.88,0,0.00,75.60,51.50,2024-06-21T20:02:00.000Z
1970-01-03 13:15:00,1.721469e+09,138,0.00,0.00,5.82,73.2,44,0.0,0.00,0.00,0.00,0.00,15.88,15.88,0,0.00,72.29,49.97,2024-06-21T20:02:00.000Z
1970-01-03 13:20:00,1.721559e+09,70,4.03,9.17,17.22,79.2,31,0.0,0.00,0.00,0.00,0.00,15.88,15.88,0,0.00,78.28,45.93,2024-06-21T20:02:00.000Z
1970-01-03 13:25:00,1.721649e+09,172,0.00,0.00,14.76,79.3,29,0.0,0.00,0.00,0.00,0.00,15.88,15.88,0,0.00,78.29,44.27,2024-06-21T20:02:00.000Z


## Create correlation matrix

In [ ]:
dataframe = pd. DataFrame (data = dataset. data, columns = dataset. feature_names)
dataframe ["target"] = dataset. target

#correlation matrix
matrix = dataframe.corr()
print(matrix)

## Bonferroni Correction of calculated p-values

In [ ]:
# Calculate adjusted p-values
reject, adj_pvals, _, _ = multitest.multipletests(result['p-value'], method='bonferroni')

# Add adjusted p-value to the DataFrame
result['adj.pval'] = adj_pvals

## Display correlation matrix

In [ ]:

dataset = datasets. load_iris()
dataframe = pd. DataFrame(data=dataset. data, columns=dataset. feature_names)
dataframe["target"] = dataset. target
matrix = dataframe.corr()

# plotting correlation matrix
plt.imshow(matrix, cmap='Blues')

# adding colorbar
plt.colorbar()

# extracting variable names
variables = []
for i in matrix.columns:
    variables.append(i)

# Adding labels to the matrix
plt.xticks(range(len(matrix)), variables, rotation=45, ha='right')
plt.yticks(range(len(matrix)), variables)

# Display the plot
plt.show()